In [14]:
import pandas as pd
import numpy as np
import os
import h5py
from sklearn import linear_model
import matplotlib.pyplot as plt

import deepdish as dd
import string
try:
    os.chdir('/data/MoL_clean/scripts')
except:
    pass
import util
# import GLM_helper as gh

import scipy.stats as stats
from joblib import Parallel, delayed
import multiprocessing

In [3]:
sub2subj = {"sub-01":"subj001", "sub-02":"subj002","sub-03":"subj003","sub-04":"subj005",
              "sub-05":"subj006", "sub-06":"subj007", "sub-07":"subj008", "sub-08":"subj009", 
           "sub-09":"subj010","sub-10":"subj011","sub-11":"subj013", "sub-12":"subj014", 
            "sub-13":"subj017", "sub-14":"subj018", "sub-15":"subj019", "sub-16":"subj020",
           "sub-17":"subj021", "sub-18":"subj022", "sub-19":"subj023", "sub-20":"subj024",'sub-21':'subj025',
           'sub-22':'subj026','sub-23':'subj027','sub-24':'subj029','sub-25':'subj031',
           'sub-101':'subj101', 'sub-102':'subj102', 'sub-103':'subj103', 'sub-105':'subj105','sub-107':'subj107','sub-108':'subj108'}

ses2w = {"ses-01":"W2", "ses-02":"W4D1", "ses-03":"W4D2"}


nv = 40962

subjects = ['sub-%.2d'%s for s in range(1,26)]
sessions = ['ses-%.2d'%s for s in range(1,3)]
runs = ['run-%.2d'%s for s in range(1,3)]
TR = 1.5
nTRs = {'Item':302, 'Loci':302, 'Encode':355}

nTRs_w4d2 = {'Item': 156, 'Loci': 156, 'Encode': 182}
SL_lh = list(dd.io.load('SLlist_verydense.lh.h5').values())
SL_rh = list(dd.io.load('SLlist_verydense.rh.h5').values())

SLlist = {'L':SL_lh, "R": SL_rh}
nSL_L = len(SLlist['L'])

In [4]:
# function for getting design matrix for encoding, item, and loci task
def get_design(task, num_TR, num_item=40, special=None):
    em = np.zeros((int(num_TR*1.5)+1,num_item))
    for index in range(num_item):
        if task == 'item' or task == 'loci':
            # item_task_structure: 1 sec fixation cross + 5 sec just item + 5 sec animate/inanimate judgement
            em[index*11+1:(index+1)*11,index]=1
        elif task == 'encoding':
            # encode_task_structure: 1 sec fixation cross + 12 sec encode 
            em[index*13+1:(index+1)*13,index]=1

    # refer to a bug in the experiment with the first few subjects where one of the loci trial was very long,
    # affect about 3 subjects.
    if special == 'long_loci':
        em[44:65,4] = 1
        em = np.concatenate((em[:,:5],em[:,6:]),axis=1)

    
    dm = util.regressor_to_TR(em, TR, num_TR) 
    

    return dm

In [5]:
# convolve design matrix with fMRI data
def deconv(V, design):
    regr = linear_model.LinearRegression(fit_intercept=False)
    regr.fit(design, np.nan_to_num(V.T))
    return regr.coef_

In [6]:
loci_dm = get_design('loci', nTRs['Loci'])
item_dm = get_design('item', nTRs['Item'])
loci_special = get_design('loci', nTRs['Loci'], special='long_loci')
encode_dm = get_design('encoding', nTRs['Encode'])
dms = {'Loci':loci_dm, 'Item':item_dm, 'Encode':encode_dm}

In [15]:
subjects = ['sub-%.2d'%s for s in range(1,26)]

for sub in subjects:
    for ses in sessions:
        for task in ['Loci','Item']:
            for i, run in enumerate(runs):
                    fmri_file = h5py.File('../preprocessed/%s_%s_%s_%s.h5'%(sub,ses,task,run),'r')
                    rh = np.array(fmri_file[f'{task}_{run}']['R'])
                    lh = np.array(fmri_file[f'{task}_{run}']['L'])
                    ah = np.array(fmri_file[f'{task}_{run}']['anterior_hipp'])
                    ph = np.array(fmri_file[f'{task}_{run}']['posterior_hipp'])
                    hippo = np.array(fmri_file[f'{task}_{run}']['hippo'])
                    ah_group = np.array(fmri_file[f'{task}_{run}']['anterior_hipp_group'])
                    ph_group = np.array(fmri_file[f'{task}_{run}']['posterior_hipp_group'])
                    hippo_group = np.array(fmri_file[f'{task}_{run}']['hippo_group'])
                    dm = dms[task]
                    if task =='Loci':
                        if (sub == 'sub-02' and ses == 'ses-01') or (sub == 'sub-04' and ses == 'ses-01') or (sub == 'sub-03' and ses=='ses-01' and i == 0): 
                            dm = loci_special                        
                    lh_beta = deconv(lh, dm)
                    rh_beta = deconv(rh, dm)          
                    ah_beta = deconv(ah, dm)         
                    ph_beta =  deconv(ph, dm)   
                    hippo_beta = deconv(hippo, dm)
                    ah_group_beta = deconv(ah_group, dm)
                    ph_group_beta = deconv(ph_group, dm)
                    hippo_group_beta = deconv(hippo_group, dm)  
                    np.savetxt('../outputs/betas/%s/%s_%s_%s_lh_beta.txt'%(task.lower(),sub,ses,run), lh_beta)
                    np.savetxt('../outputs/betas/%s/%s_%s_%s_rh_beta.txt'%(task.lower(),sub,ses,run), rh_beta)
                    np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ah_beta)
                    np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ph_beta)
                    np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_beta.txt'%(task.lower(),sub,ses,run), hippo_beta)
                    np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ah_group_beta)
                    np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ph_group_beta)
                    np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_group_beta.txt'%(task.lower(),sub,ses,run), hippo_group_beta)


        task = 'Encode'
        try: 
            run = 'run-02'
            fmri_file = h5py.File('../preprocessed/%s_%s_%s_%s.h5'%(sub,ses,task,run),'r')
        except:
             run = 'run-01'
             fmri_file = h5py.File('../preprocessed/%s_%s_%s_%s.h5'%(sub,ses,task,run),'r')
        rh = np.array(fmri_file[f'{task}_{run}']['R'])
        lh = np.array(fmri_file[f'{task}_{run}']['L'])
        ah = np.array(fmri_file[f'{task}_{run}']['anterior_hipp'])
        ph = np.array(fmri_file[f'{task}_{run}']['posterior_hipp'])
        lh_beta = deconv(lh, dms[task])
        rh_beta = deconv(rh, dms[task])
        ah_beta = deconv(ah, dms[task])         
        ph_beta =  deconv(ph, dms[task])   
        hippo_beta = deconv(hippo, dm)
        ah_group_beta = deconv(ah_group, dm)
        ph_group_beta = deconv(ph_group, dm)
        hippo_group_beta = deconv(hippo_group, dm)  
        
        np.savetxt('../outputs/betas/%s/%s_%s_%s_lh_beta.txt'%(task.lower(),sub,ses,run), lh_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_rh_beta.txt'%(task.lower(),sub,ses,run), rh_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ah_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ph_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_beta.txt'%(task.lower(),sub,ses,run), hippo_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ah_group_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ph_group_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_group_beta.txt'%(task.lower(),sub,ses,run), hippo_group_beta)


In [16]:
def get_design_recall(recall_sheet, num_TR):
    # create event matrix for retrieval. 
    recall_em = np.zeros((int(num_TR*1.5),len(recall_sheet)))
    # correct_recall_em = recall_em.copy()
    # incorrect_recall_em = recall_em.copy()
    used_idx = []
    for i in range(len(recall_sheet)):
        recall_em[int(np.nan_to_num(recall_sheet['start_time'][i])):int(np.nan_to_num(recall_sheet['end_time'][i])),i]+=1
        # if int(np.nan_to_num(recall_sheet['start_time'][i])) == int(np.nan_to_num(recall_sheet['end_time'][i])):
        #     used_idx.append(i)
    non_zero_columns = ~(recall_em==0).all(axis=0)

    recall_em = recall_em[:,non_zero_columns]

    dm = util.regressor_to_TR(recall_em, TR, num_TR) 
    
    return dm


for sub in subjects:
    for ses in sessions:
        task = 'Retrieve'
        run = 'run-01'
        fmri_file = h5py.File('../preprocessed/%s_%s_%s_%s.h5'%(sub,ses,task,run),'r')
        rh = np.array(fmri_file[task+'_'+run]['R'])
        lh = np.array(fmri_file[task+'_'+run]['L'])
        ah = np.array(fmri_file[f'{task}_{run}']['anterior_hipp'])
        ph = np.array(fmri_file[f'{task}_{run}']['posterior_hipp'])
        hippo = np.array(fmri_file[f'{task}_{run}']['hippo'])
        ah_group = np.array(fmri_file[f'{task}_{run}']['anterior_hipp_group'])
        ph_group = np.array(fmri_file[f'{task}_{run}']['posterior_hipp_group'])
        hippo_group = np.array(fmri_file[f'{task}_{run}']['hippo_group'])
        recall_sheet = pd.read_excel('../sheets/%s_recallperformance.xlsx'%sub2subj[sub], sheet_name=ses2w[ses].lower())
        
        lh_beta = deconv(lh, get_design_recall(recall_sheet, lh.shape[1]))
        rh_beta = deconv(rh, get_design_recall(recall_sheet, rh.shape[1]))

        
        ah_beta = deconv(ah,get_design_recall(recall_sheet, ah.shape[1])) 
        ph_beta = deconv(ph,get_design_recall(recall_sheet, ph.shape[1])) 
        hippo_beta = deconv(hippo,get_design_recall(recall_sheet, hippo.shape[1]))
        ah_group_beta = deconv(ah_group,get_design_recall(recall_sheet, ah_group.shape[1]))
        ph_group_beta = deconv(ph_group,get_design_recall(recall_sheet, ph_group.shape[1]))
        hippo_group_beta = deconv(hippo_group,get_design_recall(recall_sheet, hippo_group.shape[1]))

        np.savetxt('../outputs/betas/%s/%s_%s_%s_lh_beta.txt'%(task.lower(),sub,ses,run), lh_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_rh_beta.txt'%(task.lower(),sub,ses,run), rh_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ah_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ph_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_beta.txt'%(task.lower(),sub,ses,run), hippo_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ah_group_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ph_group_beta)
        np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_group_beta.txt'%(task.lower(),sub,ses,run), hippo_group_beta)



In [18]:
nTRs_w4d2 = {'Item': 156, 'Loci': 156, 'Encode': 182}

loci_dm_w4d2 = get_design('loci', nTRs_w4d2['Loci'],20)
item_dm_w4d2  = get_design('item', nTRs_w4d2['Item'],20)
encode_dm_w4d2  = get_design('encoding', nTRs_w4d2['Encode'],20)
dms_w4d2 =  {'Loci':loci_dm_w4d2, 'Item':item_dm_w4d2, 'Encode':encode_dm_w4d2}

In [19]:
subjects = ['sub-%.2d'%s for s in range(1,26)]

for sub in subjects:
    ses = 'ses-03'
    for task in ['Loci','Item']:
        for i, run in enumerate(runs):
                fmri_file = h5py.File('../preprocessed/%s_%s_%s_%s.h5'%(sub,ses,task,run),'r')
                dm = dms_w4d2[task]                     
                rh = np.array(fmri_file[f'{task}_{run}']['R'])
                lh = np.array(fmri_file[f'{task}_{run}']['L'])
                lh_beta = deconv(lh, dm)
                rh_beta = deconv(rh, dm)                 
                np.savetxt('../outputs/betas/%s/%s_%s_%s_lh_beta.txt'%(task.lower(),sub,ses,run), lh_beta)
                np.savetxt('../outputs/betas/%s/%s_%s_%s_rh_beta.txt'%(task.lower(),sub,ses,run), rh_beta)
                ah = np.array(fmri_file[f'{task}_{run}']['anterior_hipp'])
                ph = np.array(fmri_file[f'{task}_{run}']['posterior_hipp'])
                hippo = np.array(fmri_file[f'{task}_{run}']['hippo'])
                ah_group = np.array(fmri_file[f'{task}_{run}']['anterior_hipp_group'])
                ph_group = np.array(fmri_file[f'{task}_{run}']['posterior_hipp_group'])
                hippo_group = np.array(fmri_file[f'{task}_{run}']['hippo_group'])

                ah_beta = deconv(ah, dm)         
                ph_beta =  deconv(ph, dm) 
                hippo_beta = deconv(hippo, dm)
                ah_group_beta = deconv(ah_group, dm)
                ph_group_beta = deconv(ph_group, dm)
                hippo_group_beta = deconv(hippo_group, dm)

                np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ah_beta)
                np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ph_beta)
                np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_beta.txt'%(task.lower(),sub,ses,run), hippo_beta)
                np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ah_group_beta)
                np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ph_group_beta)
                np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_group_beta.txt'%(task.lower(),sub,ses,run), hippo_group_beta)
    task = 'Encode'
    try: 
        run = 'run-02'
        fmri_file = h5py.File('../preprocessed/%s_%s_%s_%s.h5'%(sub,ses,task,run),'r')
    except:
            run = 'run-01'
            fmri_file = h5py.File('../preprocessed/%s_%s_%s_%s.h5'%(sub,ses,task,run),'r')
    rh = np.array(fmri_file[f'{task}_{run}']['R'])
    lh = np.array(fmri_file[f'{task}_{run}']['L'])
    dm = dms_w4d2[task]
    lh_beta = deconv(lh, dm)
    rh_beta = deconv(rh, dm)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_lh_beta.txt'%(task.lower(),sub,ses,run), lh_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_rh_beta.txt'%(task.lower(),sub,ses,run), rh_beta)
    
    
    ah = np.array(fmri_file[f'{task}_{run}']['anterior_hipp'])
    ph = np.array(fmri_file[f'{task}_{run}']['posterior_hipp'])
    hippo = np.array(fmri_file[f'{task}_{run}']['hippo'])
    ah_group = np.array(fmri_file[f'{task}_{run}']['anterior_hipp_group'])
    ph_group = np.array(fmri_file[f'{task}_{run}']['posterior_hipp_group'])
    hippo_group = np.array(fmri_file[f'{task}_{run}']['hippo_group'])

    ah_beta = deconv(ah, dm)         
    ph_beta =  deconv(ph, dm) 
    hippo_beta = deconv(hippo, dm)
    ah_group_beta = deconv(ah_group, dm)
    ph_group_beta = deconv(ph_group, dm)
    hippo_group_beta = deconv(hippo_group, dm)

    np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ah_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ph_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_beta.txt'%(task.lower(),sub,ses,run), hippo_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ah_group_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ph_group_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_group_beta.txt'%(task.lower(),sub,ses,run), hippo_group_beta)


In [20]:
for sub in subjects:
    ses = 'ses-03'
    task = 'Retrieve'
    run = 'run-01'
    fmri_file = h5py.File('../preprocessed/%s_%s_%s_%s.h5'%(sub,ses,task,run),'r')
    rh = np.array(fmri_file[task+'_'+run]['R'])
    lh = np.array(fmri_file[task+'_'+run]['L'])
    recall_sheet = pd.read_excel('../sheets/%s_recallperformance.xlsx'%sub2subj[sub], sheet_name=ses2w[ses].lower())
    lh_beta = deconv(lh, get_design_recall(recall_sheet, lh.shape[1]))
    rh_beta = deconv(rh, get_design_recall(recall_sheet, rh.shape[1]))
    np.savetxt('../outputs/betas/%s/%s_%s_%s_lh_beta.txt'%(task.lower(),sub,ses,run), lh_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_rh_beta.txt'%(task.lower(),sub,ses,run), rh_beta)
    ah = np.array(fmri_file[f'{task}_{run}']['anterior_hipp'])
    ph = np.array(fmri_file[f'{task}_{run}']['posterior_hipp'])
    hippo = np.array(fmri_file[f'{task}_{run}']['hippo'])
    ah_group = np.array(fmri_file[f'{task}_{run}']['anterior_hipp_group'])
    ph_group = np.array(fmri_file[f'{task}_{run}']['posterior_hipp_group'])
    hippo_group = np.array(fmri_file[f'{task}_{run}']['hippo_group'])


    ah_beta = deconv(ah, get_design_recall(recall_sheet, ah.shape[1]))         
    ph_beta =  deconv(ph, get_design_recall(recall_sheet, ph.shape[1])) 
    hippo_beta = deconv(hippo, get_design_recall(recall_sheet, hippo.shape[1]))
    ah_group_beta = deconv(ah_group, get_design_recall(recall_sheet, ah_group.shape[1]))
    ph_group_beta = deconv(ph_group, get_design_recall(recall_sheet, ph_group.shape[1]))
    hippo_group_beta = deconv(hippo_group, get_design_recall(recall_sheet, hippo_group.shape[1]))

    np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ah_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_beta.txt'%(task.lower(),sub,ses,run), ph_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_beta.txt'%(task.lower(),sub,ses,run), hippo_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_anterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ah_group_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_posterior_hipp_group_beta.txt'%(task.lower(),sub,ses,run), ph_group_beta)
    np.savetxt('../outputs/betas/%s/%s_%s_%s_hippo_group_beta.txt'%(task.lower(),sub,ses,run), hippo_group_beta)

20
20
20
20
20
20
20
20
20
20
20
20
20
21
20
20
20
20
20
20
20
20
20
20
20
